In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 880kB 16.7MB/s ta 0:00:01
    100% |████████████████████████████████| 17.0MB 1.9MB/s eta 0:00:01   10% |███▎                            | 1.7MB 33.0MB/s eta 0:00:01    54% |█████████████████▌              | 9.3MB 31.9MB/s eta 0:00:01    71% |███████████████████████         | 12.2MB 32.7MB/s eta 0:00:01    79% |█████████████████████████▋      | 13.6MB 25.5MB/s eta 0:00:01    89% |████████████████████████████▌   | 15.1MB 34.6MB/s eta 0:00:01    96% |███████████████████████████████ | 16.4MB 18.0MB/s eta 0:00:01
    100% |████████████████████████████████| 26.2MB 1.2MB/s eta 0:00:01   17% |█████▊                          | 4.7MB 31.4MB/s eta 0:00:01    34% |███████████                     | 9.0MB 27.4MB/s eta 0:00:01    55% |█████████████████▊              | 14.5MB 25.5MB/s eta 0:00:01    65% |█████████████████████           | 17.2MB 26.8MB/s eta 0:00:01    98% |███████████████████████████████▋| 25.9MB 26.5MB/s eta 0:00:01
    100% |██████████████

    100% |████████████████████████████████| 153kB 15.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for cvxpy ... done
  Stored in directory: /root/.cache/pip/wheels/2b/60/0b/0c2596528665e21d698d6f84a3406c52044c7b4ca6ac737cf3
  Running setup.py bdist_wheel for plotly ... done
  Stored in directory: /root/.cache/pip/wheels/98/54/81/dd92d5b0858fac680cd7bdb8800eb26c001dd9f5dc8b1bc0ba
  Running setup.py bdist_wheel for scs ... done
  Stored in directory: /root/.cache/pip/wheels/68/3f/24/e9c75d426f600634cdac68321184ba06fdc4ab2d189b5c4541
  Running setup.py bdist_wheel for multiprocess ... done
  Stored in directory: /root/.cache/pip/wheels/b9/61/60/f2130887941f13dcbbfbc4ceef5f1114ea1ea3ff372444965c
  Running setup.py bdist_wheel for dill ... done
  Stored in directory: /root/.cache/pip/wheels/c9/de/a4/a91eec4eea652104d8c81b633f32ead5eb57d1b294eab24167
Successfully built cvxpy plotly scs multiprocess dill
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed

### Load Packages

In [2]:
import pandas as pd
import numpy as np
import helper
import project_helper

## Market Data
### Load Data
selecting large dollar volume stocks, since they are highly liquid.

In [3]:
df = pd.read_csv('data/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [4]:
project_helper.print_dataframe(close)


# Part 1: Smart Beta Portfolio


## Index Weights
The index we'll be using is based on large dollar volume stocks. 

In [5]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    cap = close*volume

    return pd.DataFrame(np.array(cap)/cap.sum(axis=1)[:,None],index=cap.index,columns=cap.columns)

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [6]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

## Portfolio Weights

In [7]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    
    dividends = dividends.cumsum()
    
    return pd.DataFrame(np.array(dividends)/dividends.sum(1)[:,None],index=dividends.index,columns = dividends.columns)

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

Tests Passed


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



### View Data

In [8]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

## Returns

In [1]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """

    return prices/prices.shift()-1

### View Data

In [10]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

## Weighted Returns
With the returns of each stock computed, use it to compute the returns for an index or ETF.

In [3]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)

    return returns*weights

### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [12]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [4]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    
    return (returns.sum(1)+1).cumprod()

### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [14]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [5]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)


    return np.std(benchmark_returns_by_date-etf_returns_by_date,ddof=1)*252**0.5


### View Data
Let's generate the tracking error using `tracking_error`.

In [16]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.10207614832007529


# Part 2: Portfolio Optimization



## Covariance


In [6]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """

    return np.cov(np.array(returns.fillna(0)).T)

### View Data


In [18]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints


### optimization


In [7]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    x = cvx.Variable(len(index_weights))
    obj = cvx.Minimize(cvx.quad_form(x,covariance_returns)+scale*cvx.norm(x-index_weights,2))
    const = [sum(x)==1, x>=0]
    prob = cvx.Problem(obj,const)
    res = prob.solve()
    
    return x.value


## Optimized Portfolio


In [20]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [21]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

Optimized ETF Tracking Error: 0.0579499564655723


## Rebalance Portfolio Over Time


In [8]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    res = list()
    ix = max(shift_size, chunk_size)
    
    while(ix < len(returns)):
        r_sub = returns.iloc[ix-chunk_size:ix]
        index_w_sub = index_weights.iloc[ix-1]
        cov_sub = get_covariance_returns(r_sub)
        res.append(get_optimal_weights(cov_sub, index_w_sub))
        ix += shift_size
    
    return res


Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [23]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio.

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [24]:
all_rebalance_weights[:-1]

[array([ 0.01043171,  0.06005971,  0.01145373,  0.00460354,  0.00899077,
         0.00659759,  0.00588507,  0.00543833,  0.01913336,  0.00440455,
         0.00117275,  0.00498858,  0.00982393,  0.02170973,  0.00656774,
         0.00762627,  0.01871597,  0.00315673,  0.01490072,  0.003     ,
         0.00180187,  0.01017081,  0.00302936,  0.00757417,  0.00280203,
         0.0037936 ,  0.00894093,  0.00609699,  0.01479787,  0.00855506,
         0.01117642,  0.00342056,  0.00574556,  0.00523573,  0.00669305,
         0.06960817,  0.00447382,  0.0037193 ,  0.00759754,  0.01160654,
         0.01144322,  0.00780366,  0.02177661,  0.02548161,  0.00738635,
         0.01096941,  0.00833655,  0.00317658,  0.01110458,  0.01594191,
         0.01052329,  0.01329638,  0.00202791,  0.01358195,  0.00451182,
         0.00312031,  0.00505538,  0.0062876 ,  0.00545544,  0.00815984,
         0.01322989,  0.00557224,  0.01257144,  0.00812511,  0.00680601,
         0.01944415,  0.0141615 ,  0.01945814,  0.0

In [9]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    sum_total_turnover = 0

    all_rebalance_weights_df = pd.DataFrame(all_rebalance_weights)
    dif_all_rebalance_weights_df = all_rebalance_weights_df - all_rebalance_weights_df.shift(-1)

    sum_total_turnover = dif_all_rebalance_weights_df.dropna().abs().as_matrix().sum()

    return sum_total_turnover * n_trading_days_in_year / (rebalance_count * shift_size)

Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [26]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

16.7267794818
